## 작업형 3

#### 아래 E-Commerce Shipping Data의 train set을 참조하여 고객이 주문한 물품의 정시 도착 여부를 예측하시오. 
#### (ID와 예측치를 csv 파일로 저장하여 제출하시오)

In [58]:
import pandas as pd
test = pd.read_csv("X_test.csv")
X = pd.read_csv("X_train.csv")
y = pd.read_csv("y_train.csv")

X_str = X[['Warehouse_block', 'Mode_of_Shipment', 'Product_importance', 'Gender']]
X_num = X[['Customer_care_calls', 'Customer_rating', 'Cost_of_the_Product', 'Prior_purchases', 'Discount_offered', 'Weight_in_gms']]
X_str = pd.get_dummies(X_str)

test_str = X[['Warehouse_block', 'Mode_of_Shipment', 'Product_importance', 'Gender']]
test_num = X[['Customer_care_calls', 'Customer_rating', 'Cost_of_the_Product', 'Prior_purchases', 'Discount_offered', 'Weight_in_gms']]
test_str = pd.get_dummies(test_str)

X_str, test_str = X_str.align(test_str, join='inner', axis=1)

from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(X_num)
X_scaled = scaler.transform(X_num)
test_scaled = scaler.transform(test_num)

X_fin = pd.concat([X_str, pd.DataFrame(X_scaled)], axis=1)
test_fin = pd.concat([test_str, pd.DataFrame(test_scaled)], axis=1)

y = y['Reached.on.Time_Y.N']

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import *
model = RandomForestClassifier()
model.fit(X_fin, y)
pred_train = model.predict_proba(X_fin)
roc_auc_score(y, pred_train[:,1])


pred_test = model.predict_proba(test_fin)
pred_test_pos = pred_test[:,1]
#print(pred_train)
roc_auc_score(y, pred_test_pos)

a = pd.DataFrame(pred_test_pos, columns=['predict_prob'])
complete = pd.concat([test['ID'], a], axis=1)
# complete = pd.DataFrame({'ID':testID, 'predict_prob':pred_test_pos})
complete.to_csv("20211204.csv", index=False)

In [36]:
param_grid={'n_estimators': range(100, 1000, 100), # 100, 200, 300, ..., 1000
            'max_features': ['auto', 'sqrt', 'log2']}
from sklearn.model_selection import GridSearchCV
grid_search=GridSearchCV(RandomForestClassifier(), param_grid, cv=5)
grid_search.fit(X_fin, y)

GridSearchCV(cv=5, estimator=RandomForestClassifier(),
             param_grid={'max_features': ['auto', 'sqrt', 'log2'],
                         'n_estimators': range(100, 1000, 100)})

In [61]:
print(grid_search.best_params_)

{'max_features': 'sqrt', 'n_estimators': 200}
